# Real Data Injection Demo (Milestone 4)
This notebook demonstrates injecting synthetic movers into **Real FITS Backgrounds**.

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import yaml
from src.injection.generator import InjectionPipeline

In [ ]:
# Load Config & Enable Real Data
with open('../config/smoke_test.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['use_real_backgrounds'] = True
config['image_size'] = [256, 256]

# Make movers bright to see them against real stars
config['magnitude_range'] = [12, 14]
config['num_objects_per_sequence'] = 5
config['noise_level']['read_noise'] = 0 # Don't add extra noise, use real noise
config['noise_level']['poisson_noise'] = False

pipeline = InjectionPipeline(config)
print("Downloading sample FITS if needed...")
frames, masks, meta = pipeline.generate_sequence()
print(f"Generated Sequence: {frames.shape}")

In [ ]:
# Visualize
plt.figure(figsize=(12, 12))
plt.title("Max Projection (Real Background + Injections)")

# Using simple percentiles for visualization scaling
proj = np.max(frames, axis=0)
vmin, vmax = np.percentile(proj, [1, 99])

plt.imshow(proj, cmap='gray', vmin=vmin, vmax=vmax)

# Annotate tracks
for m in meta['movers']:
    plt.plot([m['x0'], m['x0'] + m['vx']*10], 
             [m['y0'], m['y0'] + m['vy']*10], 
             'r-', alpha=0.5, linewidth=2)

plt.show()